# 주가 데이터 분석

## 참고

### 대시보드

대시보드 참고

- [네이버 증권 - 차트](https://finance.naver.com/item/fchart.naver?code=005930)
- [다음 증권 - 차트](https://finance.daum.net/quotes/A005930#home)
- [finvis - 차트](https://finviz.com/)
- [tradingview- 차트](https://www.tradingview.com/symbols/BTCUSD/?exchange=CRYPTO)
- [알파스퀘어 - 차트](https://www.alphasquare.co.kr/home/market-summary?code=005930)

### Plotly 공식 문서

plotly 공식 문서

- [Plotly - financial charts](https://plotly.com/python/ohlc-charts/)


### Plotly 정리 (by 쭌랩님)

공식문서를 통해 Plotly를 접하고 사용하는게 가장 정확하지만, 한글로 잘 정리 해놓은 참고자료

- [쭌랩님의 Plotly Tutorial - 파이썬 시각화의 끝판왕 마스터하기](https://wikidocs.net/book/8909)


### 기술적 지표

기술적 지표를 참고 하기에 좋은 자료 

- [알파스퀘어 기술적지표](https://alphasquare.oopy.io/board/technical-indicator)

---

In [2]:
import plotly.graph_objects as go

from ta.trend import MACD
from ta.momentum import StochasticOscillator

import numpy as np
import pandas as pd
from pykrx import stock
from pykrx import bond
from time import sleep

from datetime import datetime
from datetime import timedelta
import os
import time
from plotly.subplots import make_subplots
import glob



## 데이터 수집

In [3]:
ticker_nm = '005930' # 삼성전자 ticker
ticker_nm = '035720' # 카카오
ticker_nm = '006740'
start_date  = '20190101'
today_date1 = '20231024'

df_raw = stock.get_market_ohlcv(start_date, today_date1, ticker_nm)
df_raw = df_raw.reset_index()
df_raw['ticker'] = ticker_nm

df_raw.columns = ['date', 'open', 'high', 'low', 'close', 'volume','price_change_percentage', 'ticker']

## 보조지표 계산

### 이동평균선 계산

- 참고: [알파스퀘어 이동편균선](https://alphasquare.oopy.io/board/technical-indicator/sma)

차트의 기술적 분석을 할 때 쓰이는 가장 기본적인 보조 지표. 일정 기간 동안의 가격의 평균을 나타내며 특정 기간동안 가격의 평균을 이어서 그렸기 때문에 주식의 불규칙적인 변화를 완만하게 변화시켜서 파악하기 용이할 수 있음. 이동평균의 종류에는 크게 단순이동평균(SMA), 가중이동평균(WMA), 지수이동평균(EMA)가 있으며, 이번에 사용할 지표는 SMA. 


일반적으로 이동평균선은 5일, 20일, 60일, 120일, 240일을 사용.

- 5일: 일주일을 의미. 7일중 토,일은 빠지기 때문 
- 20일: 한달을 의미. 역시 한달 중에 주말을 제외 하면 20일이 평일 이기 때문
- 60일: 한 분기를 의미.
- 120일: 반년을 의미. 
- 240일: 1년을 의미.



#### 골든크로스, 데드크로스


이동평균선을 사용한 분석방법중 골든크로스, 데드크로스 라는 전략. 단기이동평균선이 장기 이동평균선을 돌파 하는 시점을 파악하여 매수 또는 매도 신호로 파악 하는 것

- 골든크로스: 단기 이동평균선이 장기 이동평균선 상향 돌파 (매수 신호)
- 데드크로스: 단기 이동평균선이 장기 이동평균선 하향 돌파 (매도 신호)

#### 정배열, 역배열

- 정배열: 단기 이동평균선 > 중기 이동평균선 > 장기 이동평균선
- 역배열: 단기 이동평균선 < 중기 이동평균선 < 장기 이동평균선

이에 따라 정배열이면 매수하기 적절한 시점, 역배열이면 매도하기 적절한 시점

In [4]:
df_raw['MA5'] = df_raw['close'].rolling(window=5).mean()
df_raw['MA20'] = df_raw['close'].rolling(window=20).mean()
df_raw['MA60'] = df_raw['close'].rolling(window=60).mean()
df_raw['MA120'] = df_raw['close'].rolling(window=120).mean()

### 볼린저밴드 계산


- 참고: [알파스퀘어-볼린저밴드](https://alphasquare.oopy.io/board/technical-indicator/bollinger-band)


- 볼린저 밴드의 상한선: 20일의 이동평균선 값 + 표준편차 * 2
- 볼린저 밴드의 하한선: 20일의 이동평균선 값 - 표준편차 * 2

기본적으로 볼린저 밴드는 20일의 이동평균선을 중심으로 표준편차의 두배에 해당하는 값을 상/하한선으로 설정. 표준편차의 2배에 해당하는 이유는 기본적으로 95%에 머물러야 한다는 주가의 가정이 있음. 따라서 임시적으로 이를 벗어나면 다시 이 안으로 회귀 하리라는 성질 때문.


- 밴드 하단을 상향 돌파 했을 경우 매수
- 밴드 상단을 하향 돌파 했을 경우 매도


In [5]:
std = df_raw['close'].rolling(20).std(ddof=0)

df_raw['upper'] = df_raw['MA20'] + 2 * std
df_raw['lower'] = df_raw['MA20'] - 2 * std

### MACD 계산


- 참고: [알파스퀘어 - MACD](https://alphasquare.oopy.io/board/technical-indicator/macd)

MACD는 이동평균수렴발산(Moving Average Convergence Divergence)의 약자로, 단기 이동평균선과 장기 이동평균선이 서로 가까워지거나(수렴) 멀어지는(발산) 원리를 이용하여 만들어졌다.주가 흐름의 추세를 확인하고 추세의 강도와 지속성을 파악하는데 활용되는 대표적인 추세추종형 보조지표. 

- MACD: 단기 지수이동평균 - 장기 지수이동평균
    - 단기 지수이동평균: 기본적으로 12일이다.
    - 장기 지수이동평균: 기본적으로 26일이다.
- 시그널: MACD의 K일 지수이동평균
    - K일은 기본적으로 9일을 사용한다
- 히스토그램: MACD - 시그널

이에 따른 전략은 다음과 같음

- MACD가 시그널을 상향돌파 하는 경우, 상승추세 전환으로로 판단할수 있기 때문에 매수
- MACD가 시그널을 하향돌파 하는 경우, 하락추세 전환으로로 판단할수 있기 때문에 매도


In [6]:
# MACD 
macd = MACD(close=df_raw['close'], 
            window_slow=26,
            window_fast=12, 
            window_sign=9)


df_raw['MACD_DIFF'] = macd.macd_diff()
df_raw['MACD'] = macd.macd()
df_raw['MACD_Signal'] = macd.macd_signal()

### RSI 계산

- 참고: [알파스퀘어 - RSI 계산](https://alphasquare.oopy.io/board/technical-indicator/rsi)

RSI((Relative Strength Index, 상대강도지수)는 대표적인 모멘텀 지표 중 하나로, 주가의 평균 상승폭과 하락폭을 비교하여 가격의 상승 압력과 하락 압력 간의 상대적인 강도를 나타내는 기술적 지표. 현재의 시장 상황이 과매수 상태인지 아니면 과매도 상태인지 판단하기 위해 고안되었으며, 보통 RSI 70 이상을 과매수 상태로, RSI 30 이하를 과매도 상태로 판단하는 것이 일반적.

RSI는 일정 기간 동안 주가가 전일 가겨에 비해 상승한 변화량과 하락한 변화량의 평균값을 먼저 계산. 그리고 상승한 변화량이 크면 과매수로 판단하고, 하락한 변화량이 크면 과매도로 판단하는 방식.

1. 가격이 전일 가격보다 상승한 날의 상승분은 U(up)
2. 가격이 전일 가겨보다 하락한 날의 하락분은 D(down)
3. U값과 D값의 평균값을 구하여 그것을 각각 AU(average ups)와 AD(average downs)
4. AU를 AU값으로 나눈것을 RS(relative strength)
5. 그리고 다음과 같이 RSI를 계산
  - $RSI = \frac {RS}{1+RS}$ 또는, 
  - $RSI = \frac {AU}{AU+AD}$
  
그리고 이 기간은 기본적으로 14일로 지정


RSI는 기본적으로 0~100까지의 백분율 값을 갖는다. 100에 가까울수록 최근 14일동안 주가가 상승 했으며, 0에 가까울수록 하락 했다는 해석이 강하다. 따라서 RSI가 70이상이면 과매수 국면으로, 30이하이면 과매도 국면으로 정의. 

- 과매수 구간에 머물던 RSI가 상단선을 하향 돌파시 매도
  - 즉, RSI가 70이상 이었다가 70아래로 떨어지는 시점
- 과매도 구간에 머물던 RSI가 하단선을 상향 돌파시 매수
  - 즉, RSI가 30이하 이었다가 30위로 올라가는 시점


In [14]:
df_raw['변화량'] = df_raw['close'] - df_raw['close'].shift(1)
df_raw['상승폭'] = np.where(df_raw['변화량']>=0, df_raw['변화량'], 0)
df_raw['하락폭'] = np.where(df_raw['변화량'] <0, df_raw['변화량'].abs(), 0)

# welles moving average
df_raw['AU'] = df_raw['상승폭'].ewm(alpha=1/14, min_periods=14).mean()
df_raw['AD'] = df_raw['하락폭'].ewm(alpha=1/14, min_periods=14).mean()
df_raw['RSI'] = df_raw['AU'] / (df_raw['AU'] + df_raw['AD']) * 100

### 이동평균선 차이 계산

In [15]:
df_raw['MA5-20'] = df_raw['MA5'] - df_raw['MA20']
df_raw['MA20-60'] = df_raw['MA20'] - df_raw['MA60']
df_raw['MA60-120'] = df_raw['MA60'] - df_raw['MA120']

## 보조지표 분석

In [16]:
df_raw_anal = df_raw[['date','ticker', 'close']]

### 골든크로스

In [17]:
# 골든 크로스 5-20
# 음수에서 양수로 바뀌는 모든 인덱스 찾기
idx_5_20_gold_cross = [idx for idx in range(len(df_raw)) if df_raw["MA5-20"].iloc[idx] > 0 and df_raw["MA5-20"].iloc[idx - 1] <= 0]

# 데드 크로스 5-20
# 양수에서 음수로 바뀌는 모든 인덱스 찾기
idx_5_20_dead_cross = [idx for idx in range(len(df_raw)) if df_raw["MA5-20"].iloc[idx] < 0 and df_raw["MA5-20"].iloc[idx - 1] >= 0]

# 골든 크로스 20-60
# 음수에서 양수로 바뀌는 모든 인덱스 찾기
idx_20_60_gold_cross = [idx for idx in range(len(df_raw)) if df_raw["MA20-60"].iloc[idx] > 0 and df_raw["MA20-60"].iloc[idx - 1] <= 0]

# 골든 크로스 20-60
# 음수에서 양수로 바뀌는 모든 인덱스 찾기
idx_20_60_dead_cross = [idx for idx in range(len(df_raw)) if df_raw["MA20-60"].iloc[idx] < 0 and df_raw["MA20-60"].iloc[idx - 1] >= 0]

In [18]:
df_raw_anal.loc[:, '5_20_cross'] = '-'
df_raw_anal.loc[idx_5_20_gold_cross,'5_20_cross'] = '골든크로스(매수)'
df_raw_anal.loc[idx_5_20_dead_cross,'5_20_cross'] = '데드크로스(매도)'

df_raw_anal.loc[:, '20_60_cross'] = '-'
df_raw_anal.loc[idx_20_60_gold_cross,'20_60_cross'] = '골든크로스(매수)'
df_raw_anal.loc[idx_20_60_dead_cross,'20_60_cross'] = '데드크로스(매도)'

/tmp/ipykernel_10493/1314038236.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_raw_anal.loc[:, '5_20_cross'] = '-'
/tmp/ipykernel_10493/1314038236.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_raw_anal.loc[:, '20_60_cross'] = '-'


### 정배열, 역배열

In [19]:
ascending_sq  = (df_raw['MA5-20'] > 0) & \
(df_raw['MA20-60'] > 0) & \
(df_raw['MA60-120'] > 0) 

descending_sq  = (df_raw['MA5-20'] < 0) & \
(df_raw['MA20-60'] < 0) & \
(df_raw['MA60-120'] < 0) 

In [20]:
df_raw_anal.loc[:,'array'] = '-'
df_raw_anal.loc[ascending_sq,'array'] = '정배열(매수)'
df_raw_anal.loc[descending_sq,'array'] = '역배열(매도)'

/tmp/ipykernel_10493/2497960516.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_raw_anal.loc[:,'array'] = '-'


### 볼린저밴드


In [21]:
down_reg_sq = df_raw['upper'] - df_raw['close'] 
top_reg_sq  = df_raw['lower'] - df_raw['close'] 

down_reg = [idx for idx in range(1,len(df_raw)) if down_reg_sq[idx] > 0 and down_reg_sq[idx-1] <= 0]
top_reg = [idx for idx in range(1,len(df_raw)) if top_reg_sq[idx] < 0 and top_reg_sq[idx-1] >= 0]

In [27]:
down_reg = [idx for idx in range(1,len(df_raw)) if down_reg_sq[idx] > 0 and down_reg_sq[idx-1] <= 0]
down_reg

[33,
 37,
 72,
 239,
 241,
 315,
 331,
 378,
 384,
 417,
 447,
 451,
 454,
 456,
 463,
 465,
 474,
 478,
 514,
 525,
 560,
 601,
 622,
 693,
 747,
 758,
 763,
 782,
 792,
 826,
 831,
 881,
 906,
 916,
 948,
 958,
 965,
 1013,
 1019,
 1033,
 1043,
 1052,
 1065,
 1086,
 1125,
 1129,
 1132]

In [14]:
df_raw_anal.loc[:,'Bollinger_band'] = '-'
df_raw_anal.loc[down_reg,'Bollinger_band'] = '하향회귀(매도)'
df_raw_anal.loc[top_reg,'Bollinger_band'] = '상향회귀(매수)'

/tmp/ipykernel_4092877/3325722006.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_raw_anal.loc[:,'Bollinger_band'] = '-'


### MACD

In [15]:
signal_down_cross = [idx for idx in range(1,len(df_raw)) if df_raw['MACD_DIFF'][idx] < 0 and df_raw['MACD_DIFF'][idx-1] >= 0]
signal_top_corss = [idx for idx in range(1,len(df_raw)) if df_raw['MACD_DIFF'][idx] > 0 and df_raw['MACD_DIFF'][idx-1] <= 0]


In [16]:
df_raw_anal.loc[:,'MACD'] = '-'
df_raw_anal.loc[signal_down_cross,'MACD'] = '하향돌파(매도)'
df_raw_anal.loc[signal_top_corss,'MACD'] = '상향돌파(매수)'

/tmp/ipykernel_4092877/2822066516.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_raw_anal.loc[:,'MACD'] = '-'


### RSI

In [17]:
down_reg = [idx for idx in range(1,len(df_raw)) if df_raw['RSI'][idx] > 70 and df_raw['RSI'][idx-1] <= 70]
top_reg = [idx for idx in range(1,len(df_raw)) if df_raw['RSI'][idx] < 30 and df_raw['RSI'][idx-1] >= 30]


In [18]:
df_raw_anal.loc[:,'RSI'] = '-'
df_raw_anal.loc[down_reg,'RSI'] = 'RSI 상단 하향돌파(매도)'
df_raw_anal.loc[top_reg,'RSI'] = 'RSI 하단 상향 돌파(매수)'

/tmp/ipykernel_4092877/819055055.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_raw_anal.loc[:,'RSI'] = '-'


## 날짜

In [19]:
date = '2023-01-01'


In [20]:
df_raw_anal_date = df_raw_anal[df_raw_anal['date'] > date].reset_index(drop = True)
df_raw_date = df_raw[df_raw['date'] > date].reset_index(drop = True)

In [21]:
type_list = ['매수', '매도']

In [22]:
technical_indicator = ['5_20_cross', '20_60_cross', 'array', 'Bollinger_band', 'MACD', 'RSI']
technical_indicator_row = [1, 1, 1, 1, 3, 4]


In [23]:
df_raw_anal_date

,date,ticker,close,5_20_cross,20_60_cross,array,Bollinger_band,MACD,RSI
0,2023-01-02,006740,5829,-,-,-,-,-,-
1,2023-01-03,006740,6022,-,-,-,-,-,-
2,2023-01-04,006740,6151,-,-,-,-,상향돌파(매수),-
3,2023-01-05,006740,6044,골든크로스(매수),-,정배열(매수),-,-,-
4,2023-01-06,006740,6022,-,-,정배열(매수),-,-,-
...,...,...,...,...,...,...,...,...,...
194,2023-10-18,006740,33900,데드크로스(매도),-,-,-,-,RSI 하단 상향 돌파(매수)
195,2023-10-19,006740,33900,-,-,-,-,-,-
196,2023-10-20,006740,33900,-,데드크로스(매도),-,-,-,-
197,2023-10-23,006740,33900,-,-,-,상향회귀(매수),-,-


In [24]:
type_nm = '매수'
technical_indicator_nm = '5_20_cross'

In [25]:
for type_nm in type_list:
    for technical_indicator_nm in technical_indicator:
#         print(df_raw_anal_date.index[df_raw_anal_date[technical_indicator_nm].str.contains(type_nm)])
        index_list = df_raw_anal_date.index[df_raw_anal_date[technical_indicator_nm].str.contains(type_nm)]
        for index_nm in index_list:
            print(f'{type_nm}, {technical_indicator_nm} ,{index_nm}')
        

매수, 5_20_cross ,3
매수, 5_20_cross ,164
매수, 5_20_cross ,191
매수, array ,3
매수, array ,4
매수, array ,5
매수, array ,6
매수, array ,7
매수, array ,8
매수, array ,9
매수, array ,10
매수, array ,11
매수, array ,12
매수, array ,13
매수, array ,14
매수, array ,15
매수, array ,16
매수, array ,17
매수, array ,18
매수, array ,19
매수, array ,20
매수, array ,21
매수, array ,22
매수, array ,23
매수, array ,24
매수, array ,25
매수, array ,26
매수, array ,27
매수, array ,28
매수, array ,29
매수, array ,30
매수, array ,31
매수, array ,32
매수, array ,33
매수, array ,34
매수, array ,35
매수, array ,36
매수, array ,37
매수, array ,38
매수, array ,39
매수, array ,40
매수, array ,41
매수, array ,42
매수, array ,43
매수, array ,44
매수, array ,45
매수, array ,46
매수, array ,47
매수, array ,48
매수, array ,49
매수, array ,50
매수, array ,51
매수, array ,52
매수, array ,53
매수, array ,54
매수, array ,55
매수, array ,56
매수, array ,57
매수, array ,58
매수, array ,59
매수, array ,60
매수, array ,61
매수, array ,62
매수, array ,63
매수, array ,64
매수, array ,65
매수, array ,66
매수, array ,67
매수, array ,68
매수, array ,69
매수, array ,

## 시각화

### 사용방법

전처리된 ohlcv 데이터를 통해 보조지표 옵션을 넣어주어 매수/매도 순간을 파악하는 방식

- 함수 이름: macd_vis(데이터, '보조지표형식')
- 옵션 이름
    - 5_20_cross: 이동평균선 5, 20간의 크로스를 기준으로 골든,데드크로스 전략
    - 20_60_cross: 이동평균선 20,60간의 크로스를 기준으로 골든,데드크로스 전략
    - array: 정배열,역배열을 통한 전략
    - Bollinger_band: 볼린저밴드를 통한 전략
    - MACD: MACD를 통한 전략
    - RSI: RSI를 통한 전략

In [26]:

def macd_vis(df_raw, technical_indicator_nm):
    # fig = go.Figure()
    fig = make_subplots(rows=4, cols=1, shared_xaxes=True, vertical_spacing=0.01, row_heights=[0.5, 0.1, 0.2, 0.2])

    # 캔들스틱차트
    fig.add_trace(go.Candlestick(
        x=df_raw['date'],
        open=df_raw['open'],
        high=df_raw['high'],
        low=df_raw['low'],
        close=df_raw['close'],
        increasing_line_color= 'red', decreasing_line_color= 'blue', 
        name = ''), row = 1, col = 1)

    # MA 5 
    fig.add_trace(go.Scatter(x=df_raw['date'],y=df_raw['MA5'],
                             opacity=0.7,
                             line=dict(color='blue', width=2),
                             name='MA 5') , row = 1, col = 1)

    # MA 20
    fig.add_trace(go.Scatter(x=df_raw['date'],y=df_raw['MA20'],
                             opacity=0.7,
                             line=dict(color='orange', width=2),
                             name='MA 20'), row = 1, col = 1)

    # MA 60
    fig.add_trace(go.Scatter(x=df_raw['date'],y=df_raw['MA60'],
                             opacity=0.7,
                             line=dict(color='purple', width=2),
                             name='MA 60'), row = 1, col = 1)

    # MA 120
    fig.add_trace(go.Scatter(x=df_raw['date'],y=df_raw['MA120'],
                             opacity=0.7,
                             line=dict(color='green', width=2),
                             name='MA 120'), row = 1, col = 1)

    fig.add_trace(go.Scatter(
        x=pd.concat([df_raw['date'], df_raw['date'][::-1]]),
        y=pd.concat([df_raw['upper'], df_raw['lower'][::-1]]),
        fill='toself',
        fillcolor='rgba(255,255,0,0.1)',
        line=dict(color='rgba(255,255,255,0.2)', width=2),
        name='Bollinger Band',
        showlegend=False
    ), row = 1, col = 1)

    for type_nm in type_list:
        index_list = df_raw_anal_date.index[df_raw_anal_date[technical_indicator_nm].str.contains(type_nm)]
        for index_nm in index_list:
            cross_date = df_raw['date'][index_nm]
            cross_value = df_raw['close'][index_nm]

            fig.add_annotation(x=cross_date, 
                               y=cross_value,
                               text=f'{type_nm} <br> {technical_indicator_nm}',
                               showarrow=True,
                               arrowhead=1,
                               row = 1, col = 1)                

    # Row 2 
    # volume
    colors = ['blue' if row['open'] - row['close'] >= 0 
              else 'red' for index, row in df_raw.iterrows()]

    fig.add_trace(go.Bar(x=df_raw['date'], 
                         y=df_raw['volume'],
                         marker_color=colors,
                         name = 'Volume',
                         showlegend=False
                        ), row=2, col=1)

    # MACD
    colors = ['red' if val >= 0 
              else 'blue' for val in df_raw['MACD_DIFF']]
    fig.add_trace(go.Bar(x=df_raw['date'], 
                         y=df_raw['MACD_DIFF'],
                         marker_color=colors,
                         name =  'MACD DIFF'
                        ), row=3, col=1)

    fig.add_trace(go.Scatter(x=df_raw['date'],
                             y=df_raw['MACD'],
                             line=dict(color='green', width=2),
                             name = 'MACD'
                            ), row=3, col=1)
    fig.add_trace(go.Scatter(x=df_raw['date'],
                             y=df_raw['MACD_Signal'],
                             line=dict(color='orange', width=1),
                             name = 'MACD Signal'
                            ), row=3, col=1)

    # Row 5
    # RSI
    fig.add_trace(go.Scatter(x=df_raw['date'],
                             y=df_raw['RSI'],
                             line=dict(color='red', width=1),
                             name = 'RSI'
                            ), row=4, col=1)
    
    # 수평 사각 영역 추가하기
    
    fig.add_hrect(y0=70, y1=100, line_width=0, fillcolor="white", opacity=0.1,
                  annotation_text="과매수구간", 
                  annotation_position="top right",
                  annotation_font_size=10,
                  annotation_font_color="red",
                  annotation_font_family="Times New Roman", row=4, col=1)
    
    fig.add_hrect(y0=0, y1=30, line_width=0, fillcolor="white", opacity=0.1,
                  annotation_text="과매도 구간", 
                  annotation_position="top right",
                  annotation_font_size=10,
                  annotation_font_color="blue",
                  annotation_font_family="Times New Roman", row=4, col=1)    
    
    
    # Rayout
    fig.update_layout(
        title = '삼성전자 주가',
        title_font_family="맑은고딕",
        title_font_size = 18,
        hoverlabel=dict(
            bgcolor='black',
            font_size=15,
        ),
        hovermode="x unified",
        template='plotly_dark',
        xaxis_tickangle=90,
        yaxis_tickformat = ',',
        legend = dict(orientation = 'h', xanchor = "center", x = 0.5, y= 1.2),
        barmode='group',
        margin=go.layout.Margin(
            l=10, #left margin
            r=10, #right margin
            b=10, #bottom margin
            t=100  #top margin
        ),
        height=600, width=900, 
    #     showlegend=False, 
        xaxis_rangeslider_visible=False
    )

    # update y-axis label
    fig.update_yaxes(title_text="Price", row=1, col=1)
    fig.update_yaxes(title_text="Volume", row=2, col=1)
    fig.update_yaxes(title_text="MACD", row=3, col=1)
    fig.update_yaxes(title_text="RSI", row=4, col=1)

    fig.update_xaxes(rangebreaks=[dict(bounds=["sat", "mon"])])
    return fig


In [28]:
# '5_20_cross', '20_60_cross', 'array', 'Bollinger_band', 'MACD', 'RSI'
macd_vis(df_raw_date, '20_60_cross')

/home/shjj08choi4/anaconda3/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [29]:
df_raw_date

,date,open,high,low,close,volume,price_change_percentage,ticker,MA5,MA20,...,MACD_Signal,변화량,상승폭,하락폭,AU,AD,RSI,MA5-20,MA20-60,MA60-120
0,2023-01-02,5291,6000,5269,5829,1106608,10.168210,006740,5291.2,5818.10,...,58.263881,538.0,538.0,0.0,122.985618,98.913981,55.423993,-526.90,983.133333,822.508333
1,2023-01-03,5893,6065,5764,6022,821032,3.311031,006740,5476.0,5769.70,...,49.311843,193.0,193.0,0.0,127.986645,91.848697,58.219322,-293.70,887.566667,845.191667
2,2023-01-04,6022,6151,5893,6151,322673,2.142145,006740,5682.4,5722.35,...,51.126696,129.0,129.0,0.0,128.059027,85.288076,60.023795,-39.95,788.166667,871.041667
3,2023-01-05,6172,6215,5979,6044,580404,-1.739555,006740,5867.4,5661.05,...,57.771240,-107.0,0.0,107.0,118.911954,86.838927,57.794141,206.35,677.250000,894.991667
4,2023-01-06,6022,6129,5936,6022,293145,-0.363997,006740,6013.6,5667.50,...,66.611953,-22.0,0.0,22.0,110.418243,82.207575,57.322660,346.10,631.800000,921.408333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,2023-10-18,47500,47500,33900,33900,549986,-29.958678,006740,45150.0,46395.00,...,277.717162,-14500.0,0.0,14500.0,336.967688,1273.614789,20.922101,-1245.00,1032.500000,12008.250000
195,2023-10-19,0,0,0,33900,0,0.000000,006740,42440.0,45832.50,...,-80.260322,0.0,0.0,0.0,312.898568,1182.642304,20.922101,-3392.50,412.500000,11925.083333
196,2023-10-20,0,0,0,33900,0,0.000000,006740,39680.0,45277.50,...,-500.354026,0.0,0.0,0.0,290.548670,1098.167854,20.922101,-5597.50,-191.666667,11836.500000
197,2023-10-23,0,0,0,33900,0,0.000000,006740,36800.0,44705.00,...,-936.218558,0.0,0.0,0.0,269.795193,1019.727293,20.922101,-7905.00,-820.833333,11752.333333
